In [9]:
import pandas as pd
import plotly.express as px

coral = pd.read_csv(
    "../../data/processed/CORIS_DATA/ATLANTIC/FLORIDA/Benthic_Cover_Florida_Cleaned.csv"
)
# coral = pd.read_csv(
#     "../../data/processed/CORIS_DATA/ATLANTIC/PUERTO_RICO/Benthic_Cover_Puerto_Rico_Cleaned.csv"
# )

In [10]:
coral.columns
print(sorted(coral["YEAR"].unique()))

[np.float64(nan), np.float64(2016.0), np.float64(2018.0), np.float64(2020.0), np.float64(2021.0), np.float64(2022.0)]


In [11]:
# --- clean + prep ---
d = coral.copy()

YEAR = 2022
d = d[d["YEAR"] == YEAR]

d["latitude"] = pd.to_numeric(d["latitude"], errors="coerce")
d["longitude"] = pd.to_numeric(d["longitude"], errors="coerce")
d["HARDBOTTOM_P"] = pd.to_numeric(d["HARDBOTTOM_P"], errors="coerce")

# drop rows without spatial data or percentage
d = d.dropna(subset=["latitude", "longitude", "HARDBOTTOM_P"])

# wrap longitudes if needed (safe even if already -180..180)
d["longitude"] = ((d["longitude"] + 180) % 360) - 180

In [16]:

fig = px.scatter_geo(
    d,
    lat="latitude",
    lon="longitude",
    color="HARDBOTTOM_P",
    color_continuous_scale="Viridis",   # good perceptual gradient
    range_color=(0, 100),                # percent scale (adjust if needed)
    opacity=0.7,
    hover_data={
        "HARDBOTTOM_P": True,
        "SOFTBOTTOM_P": True,
        "RUBBLE_P": True,
        "HABITAT_CD": True,
        "YEAR": True,
        "MIN_DEPTH": True,
        "MAX_DEPTH": True,
    },
)

fig.update_traces(marker=dict(size=6, opacity=0.85, line=dict(width=0.4, color="rgba(0,0,0,0.35)")))

lat_pad = 0.3
lon_pad = 0.3

lat_min, lat_max = d["latitude"].min(), d["latitude"].max()
lon_min, lon_max = d["longitude"].min(), d["longitude"].max()

fig.update_geos(
    projection_type="mercator",
    lataxis_range=[lat_min - lat_pad, lat_max + lat_pad],
    lonaxis_range=[lon_min - lon_pad, lon_max + lon_pad],
    showland=True,  landcolor="#f2f2f2",
    showocean=True, oceancolor="#dcecff",
    showcoastlines=True, coastlinecolor="#666", coastlinewidth=0.7,
    showsubunits=True, subunitcolor="#999", subunitwidth=0.5,
    showlakes=True, lakecolor="#dcecff",
    bgcolor="white",
)


fig.update_layout(
    template="plotly_white",
    title=dict(text="Hardbottom Coverage (%) — Florida (2023)", x=0.01),
    height=700,
    width=1000,
    margin=dict(l=10, r=10, t=60, b=10),
    paper_bgcolor="white",
    plot_bgcolor="white",
    coloraxis_colorbar=dict(title="Hardbottom (%)", len=0.85, thickness=18),
)

fig.write_image("../../outputs/maps/florida_hardbottom_map.png", scale = 2)
fig.write_html("../../outputs/maps/florida_hardbottom_map.html")
fig.show()